## This code will only work on the Quantopian Platform

#PipeLine Trading ALgorithm for Notebook

In [ ]:
# Imports Needed
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline
from quantopian.pipeline.data.builtin import USEquityPricing

In [ ]:
# Import Derivative from Quantopian Sector
from quantopian.pipeline.filters import Q1500US

In [ ]:
# getting the specific class of Assets or Securties 
base_universe = Q1500US()

In [ ]:
# Import data from morningstar feed 
from quantopian.pipeline.data import morningstar

In [ ]:
# vaiable with latest 
sector = morningstar.asset_classification.morningstar_sector_code.latest

In [ ]:
energy_sector = sector.eq(309)

In [ ]:
# import simple moving average and average dollor volume traded
from quantopian.pipeline.factors import SimpleMovingAverage,AverageDollarVolume

In [ ]:
dollar_volume= AverageDollarVolume(window_length = 30)

In [ ]:
# percentile between 90% and 100%
high_dollar_volume=dollar_volume.percentile_between(90,100)

In [ ]:
# top 50 in price for equities 
top_open_prices = USEquityPricing.open.latest.top(50,mask=high_dollar_volume)

In [ ]:
# highest close price between the 90 and 100 percent range
high_close_price = USEquityPricing.close.latest.percentile_between(90,100,mask= top_open_prices)

In [ ]:
def make_pipeline():
    #universe Q1500Us
    
    base_universe = Q1500US()
    
    #energy Sector 
   
    energy_sector = sector.eq(309
                             )
    #make mask for 1500US and energy
    
    base_energy = base_universe & energy_sector
    
    #dollar voloume (30 days) get the info
    
    dollar_volume = AverageDollarVolume(window_length = 30)
    
    # get the top 5% in avg dollar volume 
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    #combine the filters 
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day Mean close
    
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length = 10, mask = top_five_base_energy)
    
    # 30 Day mean close
    
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length = 30, mask = top_five_base_energy)

    #percent difference 
    
    percent_difference = ((mean_10 - mean_30)/mean_30)
    
    #list of shorts 
    
    shorts = percent_difference < 0 
    
    #list of longs
    
    longs = percent_difference > 0 
    
    #Final mask/Filter for anything to shorts for longs 
    securties_to_trade = (shorts | longs)
    #return pipeline 
    return Pipeline(columns = {
        'longs': longs,
        'shorts':shorts,
         'perc_difference': percent_difference                      
        } ,screen = securties_to_trade)
    
    
    

In [ ]:
result = run_pipeline(make_pipeline(),'2019-03-01','2019-03-01')
result

In [ ]:
result.info()

## Code for the Quantopian IDE 

In [ ]:
from quantopian.algorithm import attach_pipeline,pipeline_output
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.factors import AverageDollarVolume,SimpleMovingAverage
from quantopian.pipeline.filters.morningstar import Q1500US
from quantopian.pipeline.data import morningstar

def initialize(context):
    
    schedule_function(my_rebalance,date_rules.week_start(),time_rules.market_open(hours=1))
    
    my_pipe = make_pipeline()
    attach_pipeline(my_pipe,'my_pipeline')
    
def my_rebalance(context,data):
    for security in context.portfolio.positions:
        if security not in context.longs and security not in context.shorts and data.can_trade(security):
            order_target_percent(security,0)
            
    for security in context.longs:
        if data.can_trade(security):
            order_target_percent(security,context.long_weight)

    for security in context.shorts:
        if data.can_trade(security):
            order_target_percent(security,context.short_weight)




def my_compute_weights(context):
    
    if len(context.longs)==0:
        long_weight = 0
    else:
        long_weight = 0.5 / len(context.longs)
  
    if len(context.shorts)==0:
        short_weight = 0
    else:
        short_weight = 0.5 / len(context.shorts)
    
    return (long_weight,short_weight)






def before_trading_start(context,data):
    context.output = pipeline_output('my_pipeline')
    
    # LONG
    context.longs = context.output[context.output['longs']].index.tolist()
    
    # SHORT
    context.shorts = context.output[context.output['shorts']].index.tolist()


    context.long_weight,context.short_weight = my_compute_weights(context)
    
def make_pipeline():
    #universe Q1500Us
    
    base_universe = Q1500US()
    
    #energy Sector 
   
    energy_sector = sector.eq(309
                             )
    #make mask for 1500US and energy
    
    base_energy = base_universe & energy_sector
    
    #dollar voloume (30 days) get the info
    
    dollar_volume = AverageDollarVolume(window_length = 30)
    
    # get the top 5% in avg dollar volume 
    high_dollar_volume = dollar_volume.percentile_between(95,100)
    
    #combine the filters 
    top_five_base_energy = base_energy & high_dollar_volume
    
    # 10 day Mean close
    
    mean_10 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length = 10, mask = top_five_base_energy)
    
    # 30 Day mean close
    
    mean_30 = SimpleMovingAverage(inputs=[USEquityPricing.close], window_length = 30, mask = top_five_base_energy)

    #percent difference 
    
    percent_difference = ((mean_10 - mean_30)/mean_30)
    
    #list of shorts 
    
    shorts = percent_difference < 0 
    
    #list of longs
    
    longs = percent_difference > 0 
    
    #Final mask/Filter for anything to shorts for longs 
    securties_to_trade = (shorts | longs)
    #return pipeline 
    return Pipeline(columns = {
        'longs': longs,
        'shorts':shorts,
         'perc_difference': percent_difference                      
        } ,screen = securties_to_trade)